In [ ]:
import nbformat

# Read the Jupyter notebook
with open("./dummy_notebook.ipynb", "r", encoding="utf-8") as notebook_file:
    notebook = nbformat.read(notebook_file, as_version=4)

# Now you have the notebook content in the 'notebook' variable
# You can access cells, metadata, and other notebook components

In [ ]:
from llamabot import SimpleBot
from llamabot.prompt_manager import prompt


bot = SimpleBot(
    "You are an expert at explaining Jupyter notebook code. "
    "You will be given a notebook code cell. "
    "You will generate the necessary markdown to explain the code to a non-technical audience. "
    "Be brief and concise. "
    "Use a conversational tone, such as 'we're going to do this and that'"
)


@prompt
def provide_content(cell_source, notebook):
    """Here is the code context for you to work with:

    -----

    [[CELL SOURCE TO EXPLAIN BEGINS]]
    {{ cell_source }}
    [[CELL SOURCE TO EXPLAIN END]]

    -----

    Here is the rest of the notebook to contextualize the code cell to be explained.

    [[ALL NOTEBOOK CELLS START]]
    {% for cell in notebook.cells %}
    {{ cell.source }}
    {% endfor %}
    [[ALL NOTEBOOK CELLS END]]

    -----

    Please provide a Markdown explanation as instructed.
    """


# Create a new notebook object to store the modified notebook
new_notebook = nbformat.v4.new_notebook()

# Copy the metadata from the original notebook
new_notebook.metadata = notebook.metadata.copy()


# Iterate through the cells of the original notebook
for cell in notebook.cells:
    # make an explanation of the cell
    if cell.cell_type == "code":
        explanation = bot(provide_content(cell.source, notebook)).content
        # Create a new markdown cell with the explanation
        explanation_cell = nbformat.v4.new_markdown_cell(source=explanation)

        new_notebook.cells.append(explanation_cell)
    new_notebook.cells.append(cell)

In [ ]:
# Write the new notebook to disk
output_path = "explained_notebook.ipynb"
with open(output_path, "w") as f:
    nbformat.write(new_notebook, f)

print(f"Explained notebook has been saved to {output_path}")